In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch==2.3.0 torchtext==0.18.0 torchvision==0.18.0 torchaudio==2.3.0 torchdata==0.11.0

In [3]:
#CoLA 사용 케이스 지속적인 오류로 실행하지 못함


# 1. 라이브러리 및 데이터셋 준비
import torch
from torchtext.datasets import CoLA
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

# 2. 데이터 처리 함수
def collator(batch, tokenizer, device): # collator 함수 정의
    source, labels, texts = zip(*batch) # batch 데이터를 source, labels, texts로 언패킹
    tokenized = tokenizer( # texts를 토큰화하고 패딩/절단 처리
        texts,
        padding="longest", # 가장 긴 문장을 기준으로 패딩
        truncation=True, # 모델 최대 길이 초과 시 절단
        return_tensors="pt" # PyTorch 텐서로 반환
    )
    input_ids = tokenized["input_ids"].to(device) # input_ids를 device(GPU/CPU)로 이동
    attention_mask = tokenized["attention_mask"].to(device) # attention_mask를 device로 이동
    labels = torch.tensor(labels, dtype=torch.long).to(device) # labels를 텐서로 변환하여 device로 이동
    return input_ids, attention_mask, labels # 처리된 데이터 반환

# 3. 데이터셋 로드 및 토크나이저 설정
train_data = list(CoLA(split="train")) # CoLA 학습 데이터 로드
valid_data = list(CoLA(split="dev")) # CoLA 검증 데이터 로드
test_data = list(CoLA(split="test")) # CoLA 테스트 데이터 로드
tokenizer = AutoTokenizer.from_pretrained("gpt2") # gpt2 토크나이저 불러오기
tokenizer.pad_token = tokenizer.eos_token # 패딩 토큰을 eos 토큰으로 설정

# 4. 하이퍼파라미터 및 데이터로더 설정
epochs = 3 # 총 3 에폭 학습
batch_size = 16 # 배치 크기 설정
device = "cuda" if torch.cuda.is_available() else "cpu" # GPU 사용 가능 여부 확인
train_dataloader = DataLoader( # 학습 데이터로더 생성
    train_data,
    batch_size=batch_size,
    collate_fn=lambda x: collator(x, tokenizer, device), # collator 함수 적용
    shuffle=True,) # 데이터 셔플
valid_dataloader = DataLoader( # 검증 데이터로더 생성
    valid_data, batch_size=batch_size, collate_fn=lambda x: collator(x, tokenizer, device)) # collator 함수 적용
test_dataloader = DataLoader( # 테스트 데이터로더 생성
    test_data, batch_size=batch_size, collate_fn=lambda x: collator(x, tokenizer, device)) # collator 함수 적용
print("Train Dataset Length :", len(train_data)) # 데이터셋 크기 출력
print("Valid Dataset Length :", len(valid_data))
print("Test Dataset Length :", len(test_data))


# 5. 모델 및 옵티마이저 설정
from torch import optim
from transformers import GPT2ForSequenceClassification
model = GPT2ForSequenceClassification.from_pretrained( # GPT2 모델 불러오기
    pretrained_model_name_or_path="gpt2",
    num_labels=2).to(device) # 2개의 라벨(문법적/비문법적)로 분류하도록 설정
model.config.pad_token_id = model.config.eos_token_id # 모델의 패딩 토큰 ID 설정
optimizer = optim.Adam(model.parameters(), lr=5e-5) # Adam 옵티마이저 사용, 학습률 5e-5


# 6. 학습 및 평가 함수
import numpy as np
from torch import nn
def calc_accuracy(preds, labels): # 정확도 계산 함수 정의
    pred_flat = np.argmax(preds, axis=1).flatten() # 예측값을 1차원 배열로 변환
    labels_flat = labels.flatten() # 라벨을 1차원 배열로 변환
    return np.sum(pred_flat == labels_flat) / len(labels_flat) # 정확도 반환

def train(model, optimizer, dataloader): # 학습 함수 정의
    model.train() # 모델을 학습 모드로 전환
    train_loss = 0.0

    for input_ids, attention_mask, labels in dataloader:
        outputs = model( # 모델에 데이터 입력
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss # 손실(loss) 계산
        train_loss += loss.item() # 누적 손실 계산
        optimizer.zero_grad() # 그래디언트 초기화
        loss.backward() # 역전파
        optimizer.step() # 옵티마이저를 통해 가중치 업데이트

    train_loss = train_loss / len(dataloader) # 평균 손실 계산
    return train_loss

def evaluation(model, dataloader): # 평가 함수 정의
    with torch.no_grad(): # 그래디언트 계산 비활성화
        model.eval() # 모델을 평가 모드로 전환
        val_loss, val_accuracy = 0.0, 0.0

        for input_ids, attention_mask, labels in dataloader:
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits # 예측값(logits) 가져오기
            loss = outputs.loss # 손실(loss) 계산
            logits = logits.detach().cpu().numpy() # 텐서를 CPU로 이동 후 numpy로 변환
            label_ids = labels.to("cpu").numpy() # 라벨을 CPU로 이동 후 numpy로 변환
            accuracy = calc_accuracy(logits, label_ids) # 정확도 계산
            val_loss += loss.item() # 누적 손실 계산
            val_accuracy += accuracy # 누적 정확도 계산

    val_loss = val_loss/len(dataloader) # 평균 손실 계산
    val_accuracy = val_accuracy/len(dataloader) # 평균 정확도 계산
    return val_loss, val_accuracy


# 7. 모델 학습 및 저장
best_loss = 10000
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader) # 학습 함수 호출
    val_loss, val_accuracy = evaluation(model, valid_dataloader) # 평가 함수 호출
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Val Accuracy {val_accuracy:.4f}")

    if val_loss < best_loss: # 손실이 감소했을 경우
        best_loss = val_loss
        torch.save(model.state_dict(), "../models/GPT2ForSequenceClassification.pt") # 모델 가중치 저장
        print("Saved the model weights")

# 8. 테스트 데이터셋으로 최종 평가
model = GPT2ForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="gpt2",
    num_labels=2).to(device)
model.config.pad_token_id = model.config.eos_token_id
model.load_state_dict(torch.load("../models/GPT2ForSequenceClassification.pt")) # 저장된 가중치 불러오기
test_loss, test_accuracy = evaluation(model, test_dataloader) # 테스트 데이터로 최종 평가
print(f"Test Loss : {test_loss:.4f}")
print(f"Test Accuracy : {test_accuracy:.4f}")

ModuleNotFoundError: No module named 'torchdata.datapipes'